# Visualization for tmm2022

# 1 visualization for long term evaluation

In [5]:
import os
import numpy as np
import sys
import cv2
sys.path.append('/home/lz/TMM2022/experiment_metric/metric')
from Tracker import Tracking
from Sequence import Sequence_t
from PrRe import PrRe
from Iou import estimateIOU
print('ok')

ok


In [34]:
def trackingresult(trajectory: Tracking, sequence: Sequence_t):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    state = True
    if not len(prebbox) == len(sequence.gt) - 1:
        state =  False


    return prebbox, state

'''
    divide function
    return gt value: np.array, [0, first invisible frame]:list, [first invisible frame, end]:list
'''
def divide(sequence:Sequence_t):
    gtvalue = sequence.gt
    fullocclusion_tag = sequence.invisible    
    first_invisible = np.argwhere(fullocclusion_tag == 1)[0][0]
    before_invisible = [0, first_invisible]
    after_invisible = [first_invisible, len(gtvalue)]
    return gtvalue, before_invisible, after_invisible

def plot(sequence:Sequence_t, trackers_result,trackerlist, gtbox, before_list, after_list,):
    trackerslist = trackerlist
    gt_bbox = gtbox
    trackers_bbox = trackers_result
    before_invisible_frame = before_list
    after_invisible_frame = after_list
    colorpath = os.path.join(sequence._path, 'color')
    depthpath = os.path.join(sequence._path, 'depth')
    colorimageindex = os.listdir(colorpath)
    colorimageindex.sort()
    colorimagepath = [os.path.join(colorpath, filename) for i, filename in enumerate(colorimageindex)]

    # line_color
    red = (0, 0, 255)
    green = (0, 255, 0)
    grey = (128, 128, 128)
    yellow = (10, 215, 255)
    blue = (255, 0, 0)
    puple = (128, 0, 128)
    orange = (0, 69, 255)
    line_color = [red, grey, yellow, blue, puple, orange, green]

    save_path = os.path.join(savepath, sequence.name, 'color')
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for i in range(1,len(gt_bbox)):
        if i in range(before_invisible_frame[0], before_invisible_frame[1]):
            savename = os.path.join(save_path, '0_before_{}'.format(os.path.basename(colorimageindex[i])))
        else:
            savename = os.path.join(save_path, '1_after_{}'.format(os.path.basename(colorimageindex[i])))
        originimage = cv2.imread(colorimagepath[i])
        # gt bbox plot
        try:
            cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), line_color[-1])
        except:
            savename = savename.split(".")[0]+'_gtloss' + '.' + savename.split(".")[1]
            

        for j in range(len(trackerslist)):
            try:
                left_x = trackers_bbox[j][i-1][0]
                left_y = trackers_bbox[j][i-1][1]
                width =  trackers_bbox[j][i-1][2]
                height = trackers_bbox[j][i-1][3]
            except Exception as e:
                print(e)

            try:
                cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x+width), int(left_y)), line_color[j])
                cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x), int(left_y+height)), line_color[j])
                cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x+width), int(left_y)), line_color[j])
                cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x), int(left_y+height)), line_color[j])
            except:
                savename = savename.split(".")[0]+'_loss_{}'.format(trackerslist[j]) + '.' +savename.split(".")[1]
                continue    
        
        cv2.imwrite(savename, originimage)


savepath= '/ssd3/lz/TMM2022/visualization/longterm/'
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
trackers_list = ['iiau_rgbd','TALGD','DAL','DeT','CA3DMS','CSRKCF']
# trackers_list = ['iiau_rgbd']
all_trackers = [Tracking(tracker) for tracker in trackers_list]
# all_trackers = [Tracking(tracker) for tracker in ['CADMS']]
all_sequence = [Sequence_t(seq) for seq in seq_list]
count_seq = 0

all_ok = ['developmentboard_indoor', '1 developmentboard_indoor',
'colacan03_indoor',
'ball20_indoor',
'glass01_indoor',
'trashcan_room_occ_1',
'box_humans_room_occ_1',
'duck03_wild',
'stick_indoor',
'humans_corridor_occ_2_B',
'box_room_occ_2',
'human_entry_occ_2',
'ball10_wild',
'humans_corridor_occ_2_A',
'box_room_occ_1',
'bag02_indoor'
]
for sequence in all_sequence:
    skip_flag = 0
    if np.sum(sequence.invisible==1) == 0:
        continue

    print('{} {} start'.format(count_seq,sequence.name))
    if sequence.name in all_ok:
        continue
    gt_box, beinvis_list, afinvis_list = divide(sequence)
    trackers_box = []
    for i, trackers in enumerate(all_trackers):
        
        if sequence.name in trackers._seqlist:
            prebox, state = trackingresult(trackers, sequence)
            trackers_box.append(prebox)
            if not state:
                skip_flag = 1
                break
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            skip_flag = 1
            print('{} lack {}'.format(trackers.name, sequence.name))
            break
    if skip_flag == 1:
        continue
    plot(sequence,trackers_box,trackers_list,gt_box,beinvis_list, afinvis_list)
    count_seq += 1
    print('{} {} finished'.format(count_seq,sequence.name))

0 developmentboard_indoor start
0 colacan03_indoor start
0 ball20_indoor start
0 glass01_indoor start
0 trashcan_room_occ_1 start
0 box_humans_room_occ_1 start
0 duck03_wild start
0 cube05_indoor start
DAL lack cube05_indoor
0 stick_indoor start
0 humans_corridor_occ_2_B start
0 box_room_occ_2 start
0 human_entry_occ_2 start
0 ball10_wild start
0 humans_corridor_occ_2_A start
0 box_room_occ_1 start
0 bag02_indoor start
0 file01_indoor start
CA3DMS lack file01_indoor
0 boxes_room_occ_1 start
1 boxes_room_occ_1 finished
1 tennis_ball start
2 tennis_ball finished
2 ball01_wild start
3 ball01_wild finished
3 roller_indoor start
4 roller_indoor finished
4 thermos_office_occ_1 start
5 thermos_office_occ_1 finished
5 bottle_box start
6 bottle_box finished
6 pigeon02_wild start
7 pigeon02_wild finished
7 two_tennis_balls start
8 two_tennis_balls finished
8 trashcans_room_occ_1_B start
9 trashcans_room_occ_1_B finished
9 boxes_backpack_room_occ_1 start
10 boxes_backpack_room_occ_1 finished
10 h

## 2. Depth quality visualization

In [ ]:
'''
    depthQ: depth quality value for sequence np.array()
    return: quality index [True, False,True...]
'''
def divide_quality(sequence:Sequence_t, depthQ):
    gtvalue = sequence.gt
    depthvalue = depthQ    
    highquality = depthvalue < 0.4
    lowquality = depthvalue > 0.8
    set1 = depthvalue < 0.8
    set2 = depthvalue > 0.4
    mediumquality = set1 == set2
    return gtvalue, highquality, mediumquality, lowquality

def trackingresult(trajectory: Tracking, sequence: Sequence_t):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    state = True
    if not len(prebbox) == len(sequence.gt) - 1:
        state =  False


    return prebbox, state
def plot_quality(sequence:Sequence_t, trackers_result,trackerlist, gtbox, depthQ):
    trackerslist = trackerlist
    gt_bbox = gtbox
    trackers_bbox = trackers_result
    #highquality = high_list
    #mediumquality = medium_list
    #lowquality = low_list
    #qualityvalue = depthQ 

    colorpath = os.path.join(sequence._path, 'color')
    depthpath = os.path.join(sequence._path, 'depth')
    colorimageindex = os.listdir(colorpath)
    colorimageindex.sort()
    colorimagepath = [os.path.join(colorpath, filename) for i, filename in enumerate(colorimageindex)]

    # line_color
    red = (0, 0, 255)
    green = (0, 255, 0)
    grey = (128, 128, 128)
    yellow = (10, 215, 255)
    blue = (255, 0, 0)
    puple = (128, 0, 128)
    orange = (0, 69, 255)
    line_color = [red, grey, yellow, blue, puple, orange, green]

    save_path = os.path.join(savepath, sequence.name, 'color')
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for i in range(1,len(gt_bbox)):
        if depthQ[i] < 0.4:
            savename = os.path.join(save_path, '{}_high_{}'.format(depthQ[i], os.path.basename(colorimageindex[i])))
        elif depthQ[i] > 0.8:
            savename = os.path.join(save_path, '{}_low_{}'.format(depthQ[i], os.path.basename(colorimageindex[i])))
        else:
            savename = os.path.join(save_path, '{}_medium_{}'.format(depthQ[i], os.path.basename(colorimageindex[i])))

        originimage = cv2.imread(colorimagepath[i])
        # gt bbox plot
        try:
            cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), line_color[-1])
            cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), line_color[-1])
        except:
            savename = savename.split(".")[0]+'_gtloss' + '.' + savename.split(".")[1]
            

        for j in range(len(trackerslist)):
            left_x = trackers_bbox[j][i-1][0]
            left_y = trackers_bbox[j][i-1][1]
            width =  trackers_bbox[j][i-1][2]
            height = trackers_bbox[j][i-1][3]
            try:
                cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x+width), int(left_y)), line_color[j])
                cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x), int(left_y+height)), line_color[j])
                cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x+width), int(left_y)), line_color[j])
                cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x), int(left_y+height)), line_color[j])
            except:
                savename = savename.split(".")[0]+'_loss_{}'.format(trackerslist[j]) + '.' +savename.split(".")[1]
                continue    
        
        cv2.imwrite(savename, originimage)


'''
    depth quality 
'''
import scipy.io
mat_det = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/det.mat')
mat_cdtb = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/cdtb_25.mat')
mat_stc = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/stc_all.mat') 
det = mat_det['S'][0]
cdtb = mat_cdtb['A'][0]
stc = mat_stc['S'][0]
sequence_listQ = []
depthQ = []
for matfile in [det,stc]:
    for result in matfile:
        sequence_name = str(result[0]).split("'")[1]
        sequence_depthQ =  result[1][0]
        sequence_listQ.append(sequence_name)
        depthQ.append(sequence_depthQ)

for i in range(len(cdtb)-1):
    sequence_name = str(cdtb[i][0]).split("'")[1]
    sequence_depthQ = cdtb[i][1][0]
    sequence_listQ.append(sequence_name)
    depthQ.append(sequence_depthQ)
'''
    depth quality
'''
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
trackers_list = ['DeT', 'iiau_rgbd', 'DAL', 'TALGD', 'CA3DMS', 'CSRKCF']
all_trackers = [Tracking(tracker) for tracker in ['DeT']]
all_sequence = [Sequence_t(sequence_listQ[i]) for i in range(len(sequence_listQ))]

count_seq = 0
for index, sequence in enumerate(all_sequence):
    print('{} {} start'.format(count_seq,sequence.name))
    gt_box, high_list, medium_list, low_list = divide_quality(sequence, np.array(depthQ[index]))
    trackers_box = []
    for i, trackers in enumerate(all_trackers):
        
        if sequence.name in trackers._seqlist:
            prebox, state = trackingresult(trackers, sequence)
            if not state:
                break
            trackers_box.append(prebox)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            print('{} lack {}'.format(trackers.name, sequence.name))
            continue
    
    plot_quality(sequence,trackers_box,trackers_list,gt_box, high_list, medium_list, low_list, depthQ[index])
    count_seq += 1
    print('{} {} finished'.format(count_seq,sequence.name))


